**Set up colab for ASL data from kaggel**

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download grassknoted/asl-alphabet

In [ ]:
! unzip asl-alphabet.zip

**Import needed libraries**

In [ ]:
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
import cv2
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization,Activation,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from tensorflow.keras.applications import VGG16
from keras.models import Model
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from keras.metrics import AUC
from keras.metrics import Precision
from keras.metrics import Recall


In [ ]:
train_dir = 'asl_alphabet_train//asl_alphabet_train'
test_dir = 'asl_alphabet_test//asl_alphabet_test'
labels_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11,
               'M': 12,
               'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23,
               'Y': 24,
               'Z': 25, 'space': 26, 'del': 27, 'nothing': 28}

In [ ]:
def load_train_data():
    Y_train = []
    X_train = []
    size = 64, 64
    print("LOADING DATA FROM : ", end="")
    for folder in os.listdir(train_dir):
        print(folder, end=' | ')
        for image in os.listdir(train_dir + "/" + folder):
            # read image
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image,0)
            # resize image
            temp_img = cv2.resize(temp_img, size)
            #load converted classes
            Y_train.append(labels_dict[folder])
            X_train.append(temp_img)
    #convert X_train to numpy
    X_train = np.array(X_train)
    #normalize pixels of X_train
    X_train = X_train.astype('float32')/255.0
    #convert from 1-channel to 3-channel
    X_train = np.stack((X_train,)*3, axis=-1)
    #convert Y_train to numpy
    Y_train = np.array(Y_train)
    print()
    print("Pixels after normalize : min = %d max = %d "%(X_train.min(),X_train.max()))
    print('Loaded', len(X_train), 'images for training,', 'Train data shape =', X_train.shape)

    return X_train, Y_train


In [ ]:
def load_test_data():
    labels = []
    X_test = []
    size = 64, 64
    for image in os.listdir(test_dir):
        # read image
        temp_img = cv2.imread(test_dir + '/'+ image,0)
        # resize image
        temp_img = cv2.resize(temp_img, size)
        # load converted classes
        labels.append(labels_dict[image.split('_')[0]])
        X_test.append(temp_img)
    #convert X_test to numpy
    X_test = np.array(X_test)
    #normalize pixels of X_test
    X_test = X_test.astype('float32')/255.0
    #convert from 1-channel to 3-channel in Gray
    X_test = np.stack((X_test,)*3, axis=-1)
    #convert Y_test to numpy
    Y_test = np.array(labels)
    print("Pixels after normalize : min = %d max = %d "%(X_test.min(),X_test.max()))
    print('Loaded', len(X_test), 'images for testing,', 'Test data shape =', X_test.shape)

    return X_test, Y_test

**Load data in Gray**

In [ ]:
X_train, Y_train =  load_train_data()

LOADING DATA FROM : B | W | L | space | H | K | S | nothing | M | A | P | R | Q | E | Z | U | V | O | J | X | C | T | Y | D | N | del | I | G | F | 
Pixels after normalize : min = 0 max = 1 
Loaded 87000 images for training, Train data shape = (87000, 64, 64, 3)


In [ ]:
X_test, Y_test = load_test_data()

Pixels after normalize : min = 0 max = 1 
Loaded 29 images for testing, Test data shape = (29, 64, 64, 3)


**Preprocessing: One-hot enconding the data**

In [ ]:
print("Y_train befor one-hot encoder : ",Y_train[0])
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
print("Y_train befor one-hot encoder : ",Y_train[0])

Y_train befor one-hot encoder :  1
Y_train befor one-hot encoder :  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [ ]:
 print('Loaded', len(Y_test), 'images for testing,', 'Test data shape =', Y_test.shape)

Loaded 29 images for testing, Test data shape = (29, 29)


**Data augmentation**

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False )
datagen.fit(X_train)

#Note: number of class equal to 29 but the data in kaggel in test folder equal to 28 so you have to add manually the del_photo to test folder. 

**Create VGG16 model**

In [ ]:
model = Sequential()

model.add(VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3)))

model.add(Flatten())

model.add(Dense(420, activation='relu')) 

model.add(Dense(29, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=[AUC(name = 'Accuracy'),Precision(name = 'Precision'),Recall(name = 'Recall')] )#optimizer = sigmoed

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 420)               860580    
                                                                 
 dense_1 (Dense)             (None, 29)                12209     
                                                                 
Total params: 15,587,477
Trainable params: 15,587,477
Non-trainable params: 0
_________________________________________________________________


**Create ResNet50 Model**

In [ ]:
model_RES = Sequential()

model_RES.add(ResNet50(input_shape= (64,64,3),include_top=False,weights='imagenet'))

model_RES.add(Flatten())

model_RES.add(Dense(420, activation='relu'))

model_RES.add(Dense(29, activation='softmax'))

model_RES.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=[AUC(name = 'Accuracy'),Precision(name = 'Precision'),Recall(name = 'Recall')] )

model_RES.summary()

94781440/94765736 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 420)               3441060   
                                                                 
 dense_1 (Dense)             (None, 29)                12209     
                                                                 
Total params: 27,040,981
Trainable params: 26,987,861
Non-trainable params: 53,120
_________________________________________________________________


**Fit VGG16 model with 2 epochs**

In [ ]:
tf.config.run_functions_eagerly(True)
history = model.fit(datagen.flow(X_train,Y_train, batch_size = 64) ,epochs = 2 , validation_data = (X_test, Y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/2
1360/1360 [==============================] - 13750s 10s/step - loss: 0.3329 - Accuracy: 0.9965 - Precision: 0.9662 - Recall: 0.8749 - val_loss: 1.5314e-04 - val_Accuracy: 1.0000 - val_Precision: 1.0000 - val_Recall: 1.0000
Epoch 2/2
 406/1360 [=======>......................] - ETA: 2:41:15 - loss: 0.0268 - Accuracy: 0.9998 - Precision: 0.9932 - Recall: 0.9910

**Fit ResNet50 with 1 epochs**

In [ ]:
tf.config.run_functions_eagerly(True)
history_2 = model_RES.fit(datagen.flow(X_train,Y_train, batch_size = 64) ,epochs = 1 , validation_data = (X_test, Y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1360/1360 [==============================] - 12975s 10s/step - loss: 0.2667 - Accuracy: 0.9964 - Precision: 0.9612 - Recall: 0.9014 - val_loss: 0.0039 - val_Accuracy: 1.0000 - val_Precision: 1.0000 - val_Recall: 1.0000
